In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [24]:
import os
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [25]:
res = [.7, 0.2, 0.1]

In [26]:
import numpy as np
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
# actions = np.array(['hello', 'thanks', 'iloveyou'])
# actions = np.array(['house', 'clap', 'sign','please','learn'])
actions = np.array(['hello', 'thanks', 'iloveyou','clap', 'sign','please','learn'])

# actions = np.array(['hello'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [27]:
label_map = {label:num for num, label in enumerate(actions)}
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [28]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [30]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

d:\Yathartth\Python310\lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [31]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(X_train, y_train, epochs=300, callbacks=[tb_callback])

Epoch 1/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 8s 96ms/step - categorical_accuracy: 0.2113 - loss: 1.7335
Epoch 2/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 92ms/step - categorical_accuracy: 0.2245 - loss: 1.6107
Epoch 3/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 97ms/step - categorical_accuracy: 0.2223 - loss: 1.6198
Epoch 4/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 104ms/step - categorical_accuracy: 0.2341 - loss: 1.5786
Epoch 5/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 98ms/step - categorical_accuracy: 0.2049 - loss: 1.6931
Epoch 6/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 116ms/step - categorical_accuracy: 0.2677 - loss: 1.6351
Epoch 7/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 94ms/step - categorical_accuracy: 0.1946 - loss: 1.5619
Epoch 8/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - categorical_accuracy: 0.3468 - loss: 1.5323
Epoch 9/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 120ms/step - categorical_accuracy: 0.4328 - loss: 1.3476
Epoch 10/300
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 109ms/step - categorical_accuracy: 0.3644 - loss: 1.4632
Epoch 11/300
5/5 ━━━━━━━━━━━━━━━━━

In [32]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_12 (LSTM)                  │ (None, 30, 64)         │       442,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_13 (LSTM)                  │ (None, 30, 128)        │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_14 (LSTM)                  │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 5)              │           165 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,790,225 (6.83 MB)

 Trainable params: 596,741 (2.28 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,193,484 (4.55 MB)

In [33]:
model.save('action.h5')

In [ ]:
model.load_weights('action.h5')